In [10]:
import torch
import numpy as np
from datasets import load_dataset
import json
import os, sys
from collections import defaultdict


In [2]:
use_split = "train"
passage_ds_dir = "/mnt/nvme0/vortex_pipeline1/EVQA_passages/"
ds_dir = "/mnt/nvme0/vortex_pipeline1/EVQA_data/"
passage_ds = load_dataset('parquet', data_files ={  
                                            'train' : passage_ds_dir + 'train_passages-00000-of-00001.parquet',
                                            'test'  : passage_ds_dir + 'test_passages-00000-of-00001.parquet',
                                            })[use_split]

ds = load_dataset('parquet', data_files ={  
                                            'train' : ds_dir + 'train-00000-of-00001.parquet',
                                            'test'  : ds_dir + 'test-00000-of-00001-2.parquet',
                                            })[use_split].select(i for i in range(166900, 167000, 1)) 

passage_contents = passage_ds["passage_content"]
passage_ids = passage_ds["passage_id"]

Parameter 'indices'=<generator object <genexpr> at 0x7f8ea3913990> of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [3]:
 with open("search_result_dict.json", "r") as f:
        ranking_dict = json.load(f)

qid_list = list(ranking_dict)
ds_answers = ds["answers"]
pos_id_list = ds["pos_item_ids"]

In [14]:
Ks = [5]

recall_dict = defaultdict(list)
result_dict = defaultdict(list)

for i, (question_id, pos_ids) in enumerate(zip(qid_list, pos_id_list)):
    retrieved_docs = ranking_dict[question_id]
    retrieved_doc_scores = [doc[2] for doc in retrieved_docs]
    retrieved_docs = [doc[0] for doc in retrieved_docs]
    retrieved_doc_ids = [passage_ids[doc_idx] for doc_idx in retrieved_docs]
    retrieved_doc_texts = [passage_contents[doc_idx] for doc_idx in retrieved_docs]

    retrieved_doc_list = [
        {
            "passage_id": doc_id,
            "score": score,
        } for doc_id, score in zip(retrieved_doc_ids, retrieved_doc_scores)
    ]
    hit_list = []
    # Get answers
    
    answers = ds_answers[i]
    for retrieved_doc_text in retrieved_doc_texts:
        found = False
        for answer in answers:
            if answer.strip().lower() in retrieved_doc_text.lower():
                found = True
        if found:
            hit_list.append(1)
        else:
            hit_list.append(0)

    # print(hit_list)
    # input()
    for K in Ks:
        recall = float(np.max(np.array(hit_list[:K])))
        recall_dict[f"Pseudo Recall@{K}"].append(recall)
    
    retrieved_doc_ids = [passage_ids[doc_idx] for doc_idx in retrieved_docs] 
    hit_list = []
    for retrieved_doc_id in retrieved_doc_ids:
        found = False
        for pos_id in pos_ids:
            if pos_id == retrieved_doc_id:
                found = True
        if found:
            hit_list.append(1)
        else:
            hit_list.append(0)
    for K in Ks:
        recall = float(np.max(np.array(hit_list[:K])))
        recall_dict[f"Recall@{K}"].append(recall)

In [17]:
sum(recall_dict['Pseudo Recall@5']) / len(recall_dict['Pseudo Recall@5'])

0.85

In [19]:
sum(recall_dict['Recall@5']) / len(recall_dict['Recall@5'])

0.8